In [ ]:
import numpy as np
import copy
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import Chem
import load_smiles
import os
import scipy.io as sio
import pandas as pd

def handle_smile(smiles):
    all_smile=[]
    length=[]
    end="\n"
    element_table=["C","N","B","O","P","S","F","Cl","Br","I","(",")","=","#"]
    ring=["1","2","3","4","5","6","7","8","9","10"]
    for i in range(len(smiles)):
        #word1=sen_space[i]
        word_space=smiles[i]
        word=[]
        #word_space.insert(0,end)
        j=0
        while j<len(word_space):
            word_space1=[]
            #word_space1.append(word_space[j])
            if word_space[j]=="[":
                word_space1.append(word_space[j])
                j=j+1
                while word_space[j]!="]":
                    word_space1.append(word_space[j])
                    j=j+1
                word_space1.append(word_space[j])
                word_space2=''.join(word_space1)
                word.append(word_space2)
                j=j+1
            else:
                word_space1.append(word_space[j])

                if j+1<len(word_space):
                    word_space1.append(word_space[j+1])
                    word_space2=''.join(word_space1)
                else:
                    word_space1.insert(0,word_space[j-1])
                    word_space2=''.join(word_space1)

                if word_space2 not in element_table:
                    word.append(word_space[j])
                    j=j+1
                else:
                    word.append(word_space2)
                    j=j+2
                    
        word.append(end)
        word.insert(0,"&")
        len1=len(word)
        length.append(len1)
        all_smile.append(word)
    #print all_smile
    return all_smile

In [ ]:
def load_excel(file,target='SMILES'):      
    #read excel data
    data = pd.read_excel(file)
    
    return np.asarray(data[target])
filename = 'parent rings.xlsx'
filename1 = 'substituents.xlsx'
smiles = load_excel(filename)
smiles = handle_smile(smiles)
replace1 = load_excel(filename1,'SMILES1')
replace2 = load_excel(filename1,'SMILES2')
#replace3 = load_excel(filename1,'SMILES3')

target = np.stack([replace1,replace2],axis = 1)
def get_index(smiles):
    ele_1 = ['c','[nH]','N']
    ele_2 = [str(i) for i in range(10)]
    ele_3 = ['C']
    
    index = []
    for i in range(len(smiles)):
        if i+1 <len(smiles):
            if smiles[i+1] in ele_2:
                index.append(i+1)
            elif smiles[i] not in ele_2:
                index.append(i)
        elif smiles[i] not in ele_2:
            index.append(i)
    final_index = []
    for i,j in enumerate(index):
        smiles_copy = copy.deepcopy(smiles)
        bias1 = 0
        for p in range(2):
            smiles_copy.insert(index[i]+bias1+1,'(N)')
            bias1 = bias1+1
            check = ''.join(smiles_copy)
            m = AllChem.MolFromSmiles(check)
            if m != None:
                m1 = AllChem.MolToSmiles(m)
                m2 = AllChem.MolFromSmiles(m1)
                if m2 != None:
                    final_index.append(j)
            else:
                break
    return final_index
def replace(smiles,taraget,save_path):
    indices = []
    for i in smiles:
        indices.append(get_index(i[1:-1]))
    final_smiles = []
    n = 0
    for i,j,p in zip(smiles,indices,target):
        smile_copy = copy.deepcopy(i[1:-1])
        if len(j)>2:
            new_j = []
            for i in range(2):               
                r = np.random.randint(0,len(j)-1)
                new_j.append(j.pop(r))
            j = sorted(new_j)
        smile_copy.insert(j[0]+1,'('+p[0]+')')
        smile_copy.insert(j[1]+2,'('+p[1]+')')
        #smile_copy.insert(j[2]+3,'('+p[2]+')')
        check = ''.join(smile_copy)
        m = AllChem.MolFromSmiles(check)
        if m == None:
            if len(j)>2:
                new_j = []
                for i in range(2):               
                    r = np.random.randint(0,len(j)-1)
                    new_j.append(j.pop(r))
                j = sorted(new_j) 
            smile_copy.insert(j[0]+1,'('+p[0]+')')
            smile_copy.insert(j[1]+2,'('+p[1]+')')
            #smile_copy.insert(j[2]+3,'('+p[2]+')')
            check = ''.join(smile_copy)
            m = AllChem.MolFromSmiles(check)
            if m == None:
                print('error')
                print(n)
        final_smiles.append(check)
        n = n+1
    print(final_smiles)
    df = pd.DataFrame({'SMILES':final_smiles})
    df.to_excel(save_path)
replace(smiles,target,'search space.xlsx')